In [ ]:
%pip install llama-index-readers-file -q


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


### Custom handlers

In [ ]:
from llama_index.core.event_management.handlers import BaseEventHandler
from llama_index.core.event_management.events.query import (
    QueryEndEvent,
    QueryStartEvent,
)

In [ ]:
class MyQueryStartEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyQueryStartEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        if isinstance(event, QueryStartEvent):
            print(self.class_name())
            with open("log.txt", "a") as f:
                f.write(str(event))
                f.write("\n")

    def span_enter(self, id: str) -> None:
        """Logic for entering a span."""
        return

    def span_exit(self, id: str) -> None:
        """Logic for exiting a span."""
        return


class MyQueryEndEventHandler(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "MyQueryEndEventHandler"

    def handle(self, event) -> None:
        """Logic for handling event."""
        if isinstance(event, QueryEndEvent):
            print(self.class_name())
            with open("log.txt", "a") as f:
                f.write(str(event))
                f.write("\n")

    def span_enter(self, id: str) -> None:
        """Logic for entering a span."""
        return

    def span_exit(self, id: str) -> None:
        """Logic for exiting a span."""
        return


class QueryTracer(BaseEventHandler):
    @classmethod
    def class_name(cls) -> str:
        """Class name."""
        return "QueryTracer"

    def handle(self, event) -> None:
        """Logic for handling event."""
        return

    def span_enter(self, id: str) -> None:
        """Logic for entering a span."""
        print(f"Span: {id} entered.")

    def span_exit(self, id: str) -> None:
        """Logic for exiting a span."""
        print(f"Span: {id} exited.")

### Dispatcher


In [ ]:
import llama_index.core.event_management as evt_mgmt

dispatcher = evt_mgmt.get_dispatcher("app")

In [ ]:
dispatcher.add_handler(MyQueryStartEventHandler())
dispatcher.add_handler(MyQueryEndEventHandler())
dispatcher.add_handler(QueryTracer())

### Test It Out

In [ ]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

documents = SimpleDirectoryReader(input_dir="./data").load_data()
index = VectorStoreIndex.from_documents(documents)

In [ ]:
query_engine = index.as_query_engine(dispatcher=dispatcher)

In [ ]:
query_engine.dispatcher

Dispatcher(name='app', handlers=[<__main__.MyQueryStartEventHandler object at 0x1241fb190>, <__main__.MyQueryEndEventHandler object at 0x1241fb7c0>, <__main__.QueryTracer object at 0x1241facb0>], parent=Dispatcher(name='root', handlers=[<llama_index.core.event_management.handlers.null_handler.NullHandler object at 0x1117bfeb0>], parent=None, propagate=False), propagate=True)

In [ ]:
query_result = query_engine.query("Who is Paul?")

Span: <built-in function id> entered.
MyQueryStartEventHandler
MyQueryEndEventHandler
Span: <built-in function id> entered.


In [ ]:
query_result